# <center>CHALLENGE 4.1<center>

<p>Team Name:Group 1
<p>Student Names:Kaden Larussa, Brooks Schafer, Nick Solari
<p>Student W#s:w0762283, w0711688, w0755643

## Instructions
Use <b>generic</b> coding style unless hard-coded values are really necessary.<br>
Your code must be efficient and use self-explanatory naming.<br>
Use appropriate Python library methods for each task instead of using loops.<br>
Run your entire code and save <b>BEFORE</b> submitting. Then, submit this <b>saved</b> copy.

## Definitions

AGE_12   : Passengers with age in \[1, 13)<br>
AGE_TEEN : Passengers with age in \[13, 20)<br>
AGE_YOUNG: Passengers with age in \[20, 31)<br>
AGE_OTHER: Passengers with age >= 31

## Imports

In [64]:
import pandas as pd
import matplotlib as plt
import numpy as np
import os

## Loading Data

Load "titanic.csv" into the data frame object "data"</br>
[train.csv](https://github.com/wesm/pydata-book/blob/2nd-edition/datasets/titanic/train.csv)

In [65]:
data = pd.read_csv('titanic.csv')

## Calculate Counts

Calculate joint counts for each {Survived_i, Age_j}, i = {Yes, No}, j = {_12, _Teen, _Young, _Other}:

In [66]:
# Define age categories as per the provided specifications
def age_category(age):
    if age <= 12:
        return '_12'
    elif age > 12 and age < 20:
        return '_Teen'
    elif age >= 20 and age < 40:
        return '_Young'
    else:
        return '_Other'

data['Age_Category'] = data['Age'].apply(age_category)

data['Survived'] = data['Survived'].map({1: 'Yes', 0: 'No'})

joint_counts = pd.crosstab(data['Survived'], data['Age_Category'])

joint_counts

Age_Category,_12,_Other,_Teen,_Young
Survived,,,,
No,29,227,56,237
Yes,40,113,39,150


Calculate marginal counts for survive categories:

In [67]:

marginal_counts_survived_corrected = data['Survived'].value_counts()

marginal_counts_survived_corrected


Survived
No     549
Yes    342
Name: count, dtype: int64

Calculate marginal counts for age categories:

In [68]:
marginal_counts_age = data['Age_Category'].value_counts()

marginal_counts_age


Age_Category
_Young    387
_Other    340
_Teen      95
_12        69
Name: count, dtype: int64

Calculate total count of samples:

In [69]:

total_count_samples = data.shape[0]  

total_count_samples


891

Store all of the counts calculated above into the data frame "Count"<br>
.    Column headers: Age categories and TOTAL<br>
.    Row headers: survive categories, TOTAL<br>
Print the object Count's content:

In [70]:
joint_counts = pd.crosstab(data['Survived'], data['Age_Category'])
joint_counts.loc['TOTAL', :] = joint_counts.sum(axis=0)
joint_counts['TOTAL'] = joint_counts.sum(axis=1)
Count = joint_counts

Count


Age_Category,_12,_Other,_Teen,_Young,TOTAL
Survived,,,,,
No,29.0,227.0,56.0,237.0,549.0
Yes,40.0,113.0,39.0,150.0,342.0
TOTAL,69.0,340.0,95.0,387.0,891.0


Save Counts into the text file "Count.csv" under the folder "OUTPUT".<br>
.    Column headers: Age categories and TOTAL<br>
.    Row headers: survive categories, TOTAL:

In [71]:
# Since there is no existing "OUTPUT" folder mentioned, it will be created in the specified directory if not present.
output_directory = 'OUTPUT/'
os.makedirs(output_directory, exist_ok=True)  # Create the OUTPUT directory if it doesn't exist
output_file_path = os.path.join(output_directory, 'Count.csv')
Count.to_csv(output_file_path, header=True, index=True)  

output_file_path 


'OUTPUT/Count.csv'

## Calculate Probabilities

Generate the Probability Table T2- P() as described. Store into the data frame object "P". Print the object's content:

In [72]:
P = Count.copy()  
P = P.rename(index={'Not Survived = 0': 'Not Survived', 'Survived = 1': 'Survived'})
P.columns = ['Age_12', 'Age_Other', 'Age_Teen', 'Age_Young', 'TOTAL']
P = P[['Age_12', 'Age_Teen', 'Age_Young', 'Age_Other', 'TOTAL']] # reorder because im retarded and dont feel like going back sorry

P

,Age_12,Age_Teen,Age_Young,Age_Other,TOTAL
Survived,,,,,
No,29.0,56.0,237.0,227.0,549.0
Yes,40.0,39.0,150.0,113.0,342.0
TOTAL,69.0,95.0,387.0,340.0,891.0


Generate the Probability Table T3_1- P(Survive | Age) as described. Store into the data frame object "P_SgA". Print the object's content:

In [73]:
survivors_count = Count.loc['Yes', ['_12', '_Teen', '_Young', '_Other']]
total_age_count = Count.loc['TOTAL', ['_12', '_Teen', '_Young', '_Other']]

#calculate the conditional probability
P_SgA = survivors_count / total_age_count
P_SgA = pd.DataFrame(P_SgA).T

P_SgA.index = ['Survived | Age'] #rename
P_SgA.columns = ['Age_12', 'Age_Teen', 'Age_Young', 'Age_Other']

P_SgA.loc['Not Survived | Age'] = 1 - P_SgA.loc['Survived | Age']
P_SgA.loc['TOTAL'] = P.loc['TOTAL', ['Age_12', 'Age_Teen', 'Age_Young', 'Age_Other']] #add total row

P_SgA



,Age_12,Age_Teen,Age_Young,Age_Other
Survived | Age,0.57971,0.410526,0.387597,0.332353
Not Survived | Age,0.42029,0.589474,0.612403,0.667647
TOTAL,69.00000,95.000000,387.000000,340.000000


Generate the Probability Table T3_2- P(Survive | Age) P(Age) as described. Store into the data frame object "P_SgA_A". Print the object's content:

In [74]:
P_Age = P.loc['TOTAL', ['Age_12', 'Age_Teen', 'Age_Young', 'Age_Other']] #re extract the total row

P_Survive_Given_Age = P_SgA.drop('TOTAL')
P_SgA_A = P_Survive_Given_Age.mul(P_Age, axis=1)
P_SgA_A.loc['TOTAL'] = P_SgA_A.sum(axis=0)

P_SgA_A

,Age_12,Age_Teen,Age_Young,Age_Other
Survived | Age,40.0,39.0,150.0,113.0
Not Survived | Age,29.0,56.0,237.0,227.0
TOTAL,69.0,95.0,387.0,340.0


Generate the Probability Table T4_1- P(Age | Survive) as described. Store into the data frame object "P_AgS". Print the object's content:

In [75]:
survivors = Count.loc['Yes', :].drop('TOTAL')  #drop total column will be added later
non_survivors = Count.loc['No', :].drop('TOTAL')

#total survivors and non survivors
total_survivors = survivors.sum()
total_non_survivors = non_survivors.sum()

#calculate cond probability
P_Age_given_Survived = survivors / total_survivors
P_Age_given_Not_Survived = non_survivors / total_non_survivors

P_AgS = pd.DataFrame({
    'Age_12': [P_Age_given_Survived['_12'], P_Age_given_Not_Survived['_12']],
    'Age_Teen': [P_Age_given_Survived['_Teen'], P_Age_given_Not_Survived['_Teen']],
    'Age_Young': [P_Age_given_Survived['_Young'], P_Age_given_Not_Survived['_Young']],
    'Age_Other': [P_Age_given_Survived['_Other'], P_Age_given_Not_Survived['_Other']]
}, index=['Survived', 'Not Survived'])

P_AgS['TOTAL'] = P_AgS.sum(axis=1) #add total row again..

P_AgS


,Age_12,Age_Teen,Age_Young,Age_Other,TOTAL
Survived,0.116959,0.114035,0.438596,0.330409,1.0
Not Survived,0.052823,0.102004,0.431694,0.413479,1.0


Generate the Probability Table T4_2- P(Age | Survive) P(Survive) as described. Store into the data frame object "P_AgS_S". Print the object's content:

In [85]:
#marginal probabilities
P_Survive = Count.loc['Yes', 'TOTAL'] / total_count_samples
P_Not_Survive = Count.loc['No', 'TOTAL'] / total_count_samples

#copy the table for manipulation since im lazy
P_AgS_S = P_AgS.copy()

# get joint probabilities
P_AgS_S.loc['Survived'] *= P_Survive
P_AgS_S.loc['Not Survived'] *= P_Not_Survive

# total joint probabilities
P_AgS_S.loc['TOTAL'] = P_AgS_S.sum(axis=0)


P_AgS_S


,Age_12,Age_Teen,Age_Young,Age_Other,TOTAL
Survived,0.044893,0.043771,0.168350,0.126824,0.383838
Not Survived,0.032548,0.062851,0.265993,0.254770,0.616162
TOTAL,0.077441,0.106622,0.434343,0.381594,1.000000


## Compare

Compare the row TOTAL of T3_1- P(Survive | Age) with the row TOTAL of T2- P(). Explain:

In [77]:
print(P_SgA, '\n\n', P)

# T3_1 'TOTAL' row shows conditional probabilities of survival per age group (sum should be 1 per group).
# T2 'TOTAL' row shows actual counts of people per age group, regardless of survival status


                      Age_12   Age_Teen   Age_Young   Age_Other
Survived | Age       0.57971   0.410526    0.387597    0.332353
Not Survived | Age   0.42029   0.589474    0.612403    0.667647
TOTAL               69.00000  95.000000  387.000000  340.000000 

           Age_12  Age_Teen  Age_Young  Age_Other  TOTAL
Survived                                               
No          29.0      56.0      237.0      227.0  549.0
Yes         40.0      39.0      150.0      113.0  342.0
TOTAL       69.0      95.0      387.0      340.0  891.0


Compare the row TOTAL of T3_2- P(Survive | Age) P(Age) with the row TOTAL of T2- P(). Explain:

In [78]:
print(P_SgA_A, '\n\n', P_AgS)

# T3_2 'TOTAL' row represents the combined probability of survival and age group distribution
# T2 'TOTAL' row shows actual counts of survival status.
# T3_2's probabilities reflect theoretical likelihoods, while T2's counts are empirical data

                    Age_12  Age_Teen  Age_Young  Age_Other
Survived | Age        40.0      39.0      150.0      113.0
Not Survived | Age    29.0      56.0      237.0      227.0
TOTAL                 69.0      95.0      387.0      340.0 

                 Age_12  Age_Teen  Age_Young  Age_Other  TOTAL
Survived      0.116959  0.114035   0.438596   0.330409    1.0
Not Survived  0.052823  0.102004   0.431694   0.413479    1.0


Compare the column TOTAL of T4_1- P(Age | Survive) with the column TOTAL of T2- P(). Explain:

In [79]:
print(P_AgS, '\n\n', P)
# T4_1 'TOTAL' column shows probability distribution of age groups within each survival status (sums to 1)
# T2 'TOTAL' column shows actual counts of survivors and non-survivors, regardless of age group
# T4_1 provides theoretical likelihoods within survival categories, while T2 provides empirical data


                Age_12  Age_Teen  Age_Young  Age_Other  TOTAL
Survived      0.116959  0.114035   0.438596   0.330409    1.0
Not Survived  0.052823  0.102004   0.431694   0.413479    1.0 

           Age_12  Age_Teen  Age_Young  Age_Other  TOTAL
Survived                                               
No          29.0      56.0      237.0      227.0  549.0
Yes         40.0      39.0      150.0      113.0  342.0
TOTAL       69.0      95.0      387.0      340.0  891.0


Compare the column TOTAL of T4_2- P(Age | Survive) P(Survive) with the column TOTAL of T2- P(). Explain:

In [80]:
print(P_AgS_S, '\n\n', P)

# T4_2 'TOTAL' column represents the combined probability of each age group and survival status
# T2 'TOTAL' column shows actual counts of survivors and non-survivors across age groups
# T4_2's probabilities reflect theoretical likelihoods, while T2's counts are empirical data

                Age_12  Age_Teen  Age_Young  Age_Other     TOTAL
Survived      0.044893  0.043771   0.168350   0.126824  0.383838
Not Survived  0.032548  0.062851   0.265993   0.254770  0.616162
TOTAL         0.077441  0.106622   0.434343   0.381594  1.000000 

           Age_12  Age_Teen  Age_Young  Age_Other  TOTAL
Survived                                               
No          29.0      56.0      237.0      227.0  549.0
Yes         40.0      39.0      150.0      113.0  342.0
TOTAL       69.0      95.0      387.0      340.0  891.0


## <center> REFERENCES </center>
List resources (book, internet page, etc.) that you used to complete this challenge.